<a href="https://colab.research.google.com/github/Parsa33033/ADM_HW3/blob/main/homework3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ADM Homework 3

In [ ]:
from google.colab import drive, files
drive.mount('/content/drive', force_remount = True)

Mounted at /content/drive


A few useful libraries:

In [271]:
import requests
import os
from bs4 import BeautifulSoup
from tqdm import tqdm
from urllib.request import Request, urlopen
import pandas as pd
import csv
import re
import time
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
import pickle
import numpy as np
import collections
import heapq as hq
import matplotlib as plt
from scipy.stats.stats import pearsonr
from nltk.sentiment.vader import SentimentIntensityAnalyzer

## 1. Data collection

In [83]:
num_of_pages = 384 
directory = ""
links_file = "anime.csv"

### 1.1. Get the list of animes

We use the code down below to collect the page, name and URLs of the animes in a .csv file 

In [ ]:
anime = []

                               
if not os.path.isfile(links_file):
    with open(directory + links_file, "w", encoding='utf-8', newline='') as fobj:
        fobj.write("page,name,url\n")
        for page in tqdm(range(0, num_of_pages)):
            url = 'https://myanimelist.net/topanime.php?limit=' + str(page * 50)
            response = requests.get(url)

            soup = BeautifulSoup(response.text, 'html.parser')
            for tag in soup.find_all('tr'):
                links = tag.find_all('a')
                for link in links:        
                    if type(link.get('id')) == str and len(link.contents[0]) > 1:
                        x = (page + 1, link.contents[0], link.get('href'))
                        anime.append(x)
                        writer = csv.writer(fobj)
                        writer.writerow(x)

100%|██████████| 384/384 [02:53<00:00,  2.21it/s]


### 1.2. Crawl animes

Crawling through the URLs proved to be a problem, since after crawling through a certain number of pages with the same IP address we got security constrained: the block comes through a status code that starts with 400 (which indicates a client-side problem). To solve this issue, we checked the status code of each request while crawling through the pages. If that status code started with 400, we would stop the function for a certain amount of seconds and try again, until the status code turned to 200.


In [84]:
pages_dir = directory + "pages"
page_dir = pages_dir + "/" + "page"
article = "article"

In [ ]:
if not os.path.exists(pages_dir):
    os.makedirs(pages_dir)
    
def urlEncodeNonAscii(b):
    return re.sub('[\x80-\xFF]', lambda c: '%%%02x' % ord(c.group(0)), b)

ds = pd.read_csv(directory + links_file)
i = 1
for p, u in tqdm(zip(ds.page, ds.url)):
    d = page_dir + str(p)
    if not os.path.exists(d):
        os.makedirs(d)
    d = page_dir + str(p) + "/" + article + "_" + str(i) + ".html"
    
    if not os.path.exists(d):
#         request = requests.get(u, headers={'Cache-Control': 'no-cache', 'User-Agent': 'Mozilla/5.0'})
        req = Request(u, headers={'User-Agent': 'XYZ/3.0'})
        webpage = urlopen(req, timeout=100).read()
        with open(d, "w", encoding='utf-8') as fobj:
            fobj.write(webpage)
    i += 1

### 1.3 Parse downloaded pages

In [13]:
def getText(parent):
    return ''.join(parent.find_all(text=True, recursive=False)).strip()

In [14]:
def article_parser(article_path, soup):
        anime_title = soup.title.text.strip()
        anime_type = ""
        try:
            anime_type = getText(soup.find('span', text="Type:").parent.a)
        except:
            pass
        anime_num_episodes = ""
        try:
            anime_num_episodes = getText(soup.find('span', text="Episodes:").parent)
        except:
            pass
        anime_aired = ""
        try:
            anime_aired = getText(soup.find('span', text="Aired:").parent).split(" to ")
        except:
            pass
        anime_status = ""
        try:
            anime_status = getText(soup.find('span', text="Status:").parent)
        except:
            pass
        anime_score = ""
        try:
            anime_score = getText(soup.find('span', text="Score:").parent.find_all('span', {'class': 'score-label'})[0])
        except:
            pass
        anime_users = ""
        try:
            anime_users = getText(soup.find('span', text="Members:").parent)
        except:
            pass
        anime_rank = ""
        try:
            anime_rank = getText(soup.find('span', text="Ranked:").parent)
        except:
            pass
        anime_popularity = ""
        try:
            anime_popularity = getText(soup.find('span', text="Popularity:").parent)
        except:
            pass
        anime_description = ""
        try:
            anime_description = getText(soup.find('h2', text="Synopsis").parent.parent.p)
        except:
            pass
        anime_related = []
        try: 
            anime_related = list(set(map(getText,soup.find('h2', text="Related Anime").parent.parent.tr.find_all(lambda tag:tag.name == "a" and tag.href != "")))) 
        except: 
            pass
        anime_characters = []
        try:
            anime_characters = list(filter(None,list(map(lambda tr: re.split('\n+', tr.find_all('td')[1].text.strip())[0], soup.find('h2', text="Characters & Voice Actors").find_next('div').find_all('tr')))))
            anime_characters = [e.split(", ") for e in anime_characters]
        except:
            pass
        anime_voices = []
        try:
            anime_voices = list(filter(None,list(map(lambda tr: re.split('\n+', tr.find_all('td')[0].text.strip())[0], soup.find('h2', text="Characters & Voice Actors").find_next('div').find_all('tr')))))
            anime_voices = [e.split(", ") for e in anime_voices]
        except:
            pass
        anime_staff = []
        try:
            anime_staff = list(map(lambda tr: [re.split("\n+", tr.text.strip())[0].split(', '), re.split("\n+", tr.text.strip())[1]], soup.find('h2', text="Staff").find_next('div').find_all('tr')))
        except:
            pass
        l = [anime_title,
                anime_type,
                anime_num_episodes,
                anime_aired,
                anime_status,
                anime_score,
                anime_users,
                anime_rank,
                anime_popularity,
                anime_description,
                anime_related,
                anime_characters,
                anime_voices,
                anime_staff]
        return l
    

We create a single .tsv file containing the information for each anime. We decided to use a single .tsv file instead of one for each anime because we felt it was the best solution and we felt more comfortable working with it.

In [17]:
num_of_docs = 19125
i = 1
with open('anime.csv', 'rb') as urls:
    urls_ds = pd.read_csv(urls)
    with open('anime.tsv', 'w', encoding="utf-8", newline='') as anime:
        for p in range(1, num_of_pages + 1):
            flag = True
            if (flag): 
                article_path = page_dir + str(p) + "/" + article + "_" + str(i) + ".html"
                while os.path.isfile(article_path):
                    article_path = page_dir + str(p) + "/" + article + "_" + str(i) + ".html"
                    if os.path.isfile(article_path):
                        if i % 100 == 0:
                            print(p, "--->", i, "--->", article_path)
                        with open(article_path, 'rb') as html:
                            soup = BeautifulSoup(html, "html.parser")
                            l = article_parser(article_path, soup)
                            l.append(urls_ds.url[i - 1])
                            l.append(urls_ds.name[i - 1])
                            writer = csv.writer(anime, delimiter='\t')
                            writer.writerow(l)
                    i += 1


## 2. Search engine

We first created a better dataframe by adding the correct columns:


In [143]:
ds = pd.read_csv("anime.tsv", delimiter='\t')
ds.columns = ['anime_title',
                'anime_type',
                'anime_num_episodes',
                'anime_aired',
                'anime_status',
                'anime_score',
                'anime_users',
                'anime_rank',
                'anime_popularity',
                'anime_description',
                'anime_related',
                'anime_characters',
                'anime_voices',
                'anime_staff',
                'url',
                'title']
len(ds)

18741

In [95]:
nltk.download() #this was to install some additional modules we needed

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


True

Here we define the functions to preprocess our data. We removed stopwords and punctuation, plus we used a porter stemmer to efficiently stem every word we were dealing with

In [274]:
ps = PorterStemmer()
stopwords = stopwords.words()
def isNaN(string):
    return string != string
def preprocess(sentence):
    if sentence == "" or isNaN(sentence):
        return []
    text_tokens = nltk.word_tokenize(sentence)
    tokens_without_sw = [ps.stem(word) for word in text_tokens if not word in stopwords and word.isalnum()]
    return tokens_without_sw

### 2.1 Conjunctive query

#### 2.1.1. Create your index!

We first create an index dictionary, called preprocessed_index, save it as a pickle file, and later invert it to allow us to efficiently search through it:

In [ ]:
index_dict = {}
for idx, desc in zip(ds.index, ds.anime_description):
    if idx % 1000 == 0: 
        print("---->", idx) #this is a simple print to check the progress while building the dictionary
    preprocessed_list = preprocess(desc)
    for term in preprocessed_list:
        if term in index_dict:
            index_dict[term].append(idx)
        else:
            index_dict[term] = []
a_file = open("preprocessed_index.pkl", "wb") #we decided to save the dictionary as a pickle file
pickle.dump(index_dict, a_file)


In [29]:
a_file = open("preprocessed_index.pkl", "rb")
inverseindex_dict = pickle.load(a_file)

In [ ]:
inverse_index = {} #SHOULD I ADD THIS CELL???
for key, value in index_dict.items():
    for idx in value:
        if idx in inverseindex_dict:
            inverseindex_dict[idx].add()
        else:
            inverseindex_dict[idx] = set()

#### 2.1.2. Execute the query

We can see a simple example of how our code works down here:


In [30]:
query = input()
query = preprocess(query)
s = set(map(int, np.linspace(0, 19000, 19001)))
for q in query:
    s = s.intersection(inverseindex_dict[q])
ds.iloc[list(s), [-1, 9, -2]] #returning the correct columns

dragon


,title,anime_description,url
9728,Chaos Dragon: Sekiryuu Seneki,"In 3015, the year of Huanli, two countries, Do...",https://myanimelist.net/anime/30091/Chaos_Drag...
8194,Bada-ui Jeonseol Jangbogo,"In a not too distant future, the government se...",https://myanimelist.net/anime/10194/Bada-ui_Je...
5635,X/1999,"At the millennial edge, the concluding battle ...",https://myanimelist.net/anime/155/X_1999
16387,Kiki to Lala no Mamatte Suteki!,Lala finds a myserious egg that hatches into a...,https://myanimelist.net/anime/29249/Kiki_to_La...
1030,Hayate no Gotoku!!,Dr. Saotome returns from the dead after being ...,https://myanimelist.net/anime/4192/Hayate_no_G...
...,...,...,...
1528,Fairy Tail Movie 2: Dragon Cry,Dragon Cry is a magical artifact of deadly pow...,https://myanimelist.net/anime/30778/Fairy_Tail...
1526,Dragon Ball Kai: Mirai ni Heiwa wo! Goku no Ta...,Special episode about Trunks going back to the...,https://myanimelist.net/anime/11359/Dragon_Bal...
7674,Choegang Top Plate,The storyline is unknown but judging from the ...,https://myanimelist.net/anime/31211/Choegang_T...
17401,Harem Time The Animation,A special governmental program has been introd...,https://myanimelist.net/anime/14991/Harem_Time...


### 2.2 Conjunctive query & Ranking score

#### 2.2.1. Inverted index

In [31]:
def tf(term, desc_preprocessed):
    return desc_preprocessed.count(term)/ len(desc_preprocessed)
def idf(term, inverseindex_dict):
    return np.log(len(inverseindex_dict)/len(inverseindex_dict[term]))
def tf_idf(term, desc_preprocessed, inverseindex_dict):
    x = tf(term, desc_preprocessed) * idf(term, inverseindex_dict)
    return x

By using the previously defined functions, we create an inverted index containing the TF-IDF values:

In [ ]:
tfidf_invertedindex_dict = {}
for idx, desc in zip(ds.index, ds.anime_description):
    if idx % 1000 == 0: 
        print("---->", idx)
    preprocessed_list = preprocess(desc)
    for term in preprocessed_list:
        if term in tfidf_invertedindex_dict:
            tfidf = tf_idf(term, preprocessed_list, inverseindex_dict)
            tfidf_invertedindex_dict[term].append((idx, tfidf))
        else:
            tfidf_invertedindex_dict[term] = []
a_file = open("tfidf_invertindex.pkl", "wb")
pickle.dump(tfidf_invertedindex_dict, a_file)

In [32]:
a_file = open("tfidf_invertindex.pkl", "rb")
tfidf_invertedindex_dict = pickle.load(a_file)

In [ ]:
desc_dict = {}
for idx, desc in zip(ds.index, ds.anime_description):
    if idx % 1000 == 0: print(idx)
    desc = preprocess(desc)
    desc_vec = {}
    for d in desc:
        tfidf = 0
        for i in tfidf_invertedindex_dict[d]:
            if idx == i[0]:
                tfidf = i[1]
                break
        desc_vec[d] = tfidf 
    desc_dict[idx] = desc_vec
desc_dict_file = open("desc_dict.pkl", "wb")
pickle.dump(desc_dict, desc_dict_file)

In [34]:
desc_dict_file = open("desc_dict.pkl", "rb")
desc_dict = pickle.load(desc_dict_file)

In [35]:
def cosine_similarity_dict(dict1, dict2):
    s = set(dict1.keys()).intersection(set(dict2.keys()))
    c = 0
    norm1 = 0
    norm2 = 0
    for i in s:
        c += dict1[i] * dict2[i]
    for i in dict1.values():
        norm1 += np.power(i, 2)
    norm1 = np.sqrt(norm1)
    
    for i in dict2.values():
        norm2 += np.power(i, 2)
    norm2 = np.sqrt(norm2)
    
    return c / (norm1 * norm2)

#### 2.2.2. Execute the query

After creating our dictionary and defining a fucntion to calculate the cosine similarity, we can finally execute the input query and fetch results from our search engine. We decided to return the first 4 results.

In [244]:
query = input()
query = preprocess(query)
query_vec = {}
for q in query:
    query_vec[q] = tf_idf(q, query, inverseindex_dict)
sim_dict = {}
for idx, desc_tfidf in desc_dict.items():
    sim_dict[idx] = cosine_similarity_dict(query_vec, desc_tfidf)
sim_ds = pd.DataFrame({'similarity': sim_dict.values()})
ds['similarity'] = sim_dict.values()
k = 4 #our choice of k-results
clean_dict = {k: sim_dict[k] for k in sim_dict.keys() if sim_dict[k] != 0.0 and not np.isnan(sim_dict[k])}
clean_dict = {k: v for k, v in sorted(clean_dict.items(), key=lambda item: item[1])}

l = [i for i in clean_dict.keys()]
l = l[-k:]

heap = hq.heapify(l) #storing the top k-results in a heap
ds.iloc[l, [-1, 9, 14, 15]].sort_values('similarity', ascending=False) 
#we print the correct columns and sort them accordingly

super saiyan


<ipython-input-35-c07044aa0f89>:16: RuntimeWarning: invalid value encountered in double_scalars
  return c / (norm1 * norm2)


,similarity,anime_description,url,title
1972,0.533381,"Following the defeat of a great adversary, Gok...",https://myanimelist.net/anime/14837/Dragon_Bal...,Dragon Ball Z Movie 14: Kami to Kami
2248,0.349173,As Goku investigates the destruction of the So...,https://myanimelist.net/anime/901/Dragon_Ball_...,"Dragon Ball Z Movie 08: Moetsukiro!! Nessen, R..."
7059,0.314386,Super deformed .,https://myanimelist.net/anime/30291/Mini_Hama_...,Mini Hama: Minimum Hamatora
8786,0.296940,is a cinematic attraction at Universal Studios...,https://myanimelist.net/anime/42449/Dragon_Bal...,Dragon Ball Z: The Real 4-D at Super Tenkaichi...


### An alternative solution (way faster and simpler)

We felt that using cosine similarity wasn't possibly the best approach to find results. This is our take on a simpler and, arguably, more efficient solution to find relevant results:

In [36]:
query = input()
query = preprocess(query)
s = set(map(int, np.linspace(0, 19000, 19001)))
for q in query:
    l = [x[0] for x in tfidf_invertedindex_dict[q]]
    s = s.intersection(l)
# with open('anime.tsv', 'rb') as anime:
    

saiyan race


In [37]:
q = [x for x in tfidf_invertedindex_dict[q] if x[0] in s]
q = [x[0] for x in sorted(q, key=lambda x: x[1])]
k = int(input())
ds.iloc[q[:k]]

4


,anime_title,anime_type,anime_num_episodes,anime_aired,anime_status,anime_score,anime_users,anime_rank,anime_popularity,anime_description,anime_related,anime_characters,anime_voices,anime_staff,url,title
392,Dragon Ball Super: Broly - MyAnimeList.net,Movie,1,"['Dec 14, 2018']",Finished Airing,8.12,"278,097",#402,#631,"Forty-one years ago on Planet Vegeta, home of ...","['Dragon Ball Super', 'Dragon Ball Super: Supe...","[['Son', 'Gokuu'], ['Vegeta'], ['Broly'], ['Go...","[['Nozawa', 'Masako'], ['Horikawa', 'Ryo'], ['...","[[['Sabat', 'Christopher'], 'Producer, ADR Dir...",https://myanimelist.net/anime/36946/Dragon_Bal...,Dragon Ball Super: Broly
1014,Dragon Ball Kai (Dragon Ball Z Kai) - MyAnimeL...,TV,97,"['Apr 5, 2009', 'Mar 27, 2011']",Finished Airing,7.71,"315,238",#1036,#542,"Five years after the events of Dragon Ball, ma...","['Dragon Ball Z', 'Dragon Ball', 'Dragon Ball ...","[['Son', 'Gokuu'], ['Vegeta'], ['Son', 'Gohan'...","[['Nozawa', 'Masako'], ['Horikawa', 'Ryo'], ['...","[[['Sabat', 'Christopher'], 'Producer, ADR Dir...",https://myanimelist.net/anime/6033/Dragon_Ball...,Dragon Ball Kai
1437,Dragon Ball Z Special 1: Tatta Hitori no Saish...,Special,1,"['Oct 17, 1990']",Finished Airing,7.55,"87,557",#1469,#1785,"Bardock, Son Goku's father, is a low-ranking S...","['Dragon Ball: Episode of Bardock', 'Dragon Ba...","[['Freeza'], ['Bardock'], ['Son', 'Gokuu'], ['...","[['Nakao', 'Ryusei'], ['Nozawa', 'Masako'], ['...","[[['Nishio', 'Daisuke'], 'Director'], [['Koyam...",https://myanimelist.net/anime/986/Dragon_Ball_...,Dragon Ball Z Special 1: Tatta Hitori no Saish...


## 3. Define a new score!

We decided to define a new score that builds and improves upon the previously used cosine similarity: we assign different weights to a series of parameters, namely the cosine similarity between the query and the documents, the anime's type (TV, Movie, Music, etc.), the anime's characters and the staff that worked on it. 
The formula we used looks like this:

$$Custom score =  cos.similarity * 0.25 + type * 0.25 + characters * 0.25 + staff * 0.25$$


Type, characters and staff variables are simply calculated as 1 if any word in the query input matches a word in the respective column, and 0 if no words from the query input are found. We felt that this would be a good way to let users filter for what they are interested in in their search: for example, if I want to find a movie from the Dragon Ball serie, I just have to type "Dragon Ball Movie" and the animes classified as movies will be prioritized.

In the following cells we create the new dictionaries that will help us calculate our custom score:

In [ ]:
inverted_idx_desc = {}
inverted_idx_type = {}
inverted_idx_chars = {}
inverted_idx_staff = {}
for idx, t, chars, staff, desc in zip(ds.index, ds.anime_type, ds.anime_characters, ds.anime_staff, ds.anime_description):
    if idx % 1000 == 0: 
        print("---->", idx)
    preprocessed_list = preprocess(desc)
    for term in preprocessed_list:
        if term in inverted_idx_desc:
            tfidf = tf_idf(term, preprocessed_list, inverseindex_dict)
            inverted_idx_desc[term].append((idx, tfidf))
        else:
            inverted_idx_desc[term] = []
    
    if t in inverted_idx_type:
            inverted_idx_type[t].append(idx)
    else:
        inverted_idx_type[t] = []
            
    for char_name in chars:
        if char_name in inverted_idx_chars:
            inverted_idx_chars[char_name].append(idx)
        else:
            inverted_idx_chars[char_name] = []
        
    for staff_name in staff:
        if staff_name in inverted_idx_staff:
            inverted_idx_staff[staff_name].append(idx)
        else:
            inverted_idx_staff[staff_name] = []   
    
desc_file = open("invertindex_desc.pkl", "wb")
pickle.dump(inverted_idx_desc, desc_file)

type_file = open("invertindex_type.pkl", "wb")
pickle.dump(inverted_idx_type, type_file)

chars_file = open("invertindex_chars.pkl", "wb")
pickle.dump(inverted_idx_chars, chars_file)

staff_file = open("invertindex_staff.pkl", "wb")
pickle.dump(inverted_idx_staff, staff_file)

In [177]:
desc_file = open("invertindex_desc.pkl", "rb")
inverted_idx_desc = pickle.load(desc_file)

type_file = open("invertindex_type.pkl", "rb")
inverted_idx_type = pickle.load(type_file)

chars_file = open("invertindex_chars.pkl", "rb")
inverted_idx_chars = pickle.load(chars_file)

staff_file = open("invertindex_staff.pkl", "rb")
inverted_idx_staff = pickle.load(staff_file)

In [38]:
desc_dict_file = open("desc_dict.pkl", "rb")
desc_dict = pickle.load(desc_dict_file)

In [ ]:

for idx, t, chars, staff, desc in zip(ds.index, ds.anime_type, ds.anime_characters, ds.anime_staff, ds.anime_description):
    if idx % 1000 == 0: print(idx)
    desc = preprocess(desc)
    desc_vec = {}
    for d in desc:
        tfidf = 0
        for i in tfidf_invertedindex_dict[d]:
            if idx == i[0]:
                tfidf = i[1]
                break
        desc_vec[d] = tfidf 
    desc_dict[idx] = {'desc': desc_vec, 'chars': chars, 'type': t, 'staff': staff}
big_dict_file = open("big_dict.pkl", "wb")
pickle.dump(desc_dict, big_dict_file)

In [39]:
big_dict_file = open("big_dict.pkl", "rb")
big_dict = pickle.load(big_dict_file)

Finally, we can input a query and try out our new score:

In [318]:
query = input()
query = preprocess(query)
query_vec = {}
for q in query:
    query_vec[q] = tf_idf(q, query, inverseindex_dict)
sim_dict = {}
for idx, d in big_dict.items():
    t, chars, staff, desc = [d['type'], d['chars'], d['staff'], d['desc']]
    t = str(t)
    t = t.lower()
    
    #Searching if words contained in the query are also contained in type, characters or staff
    t_num = 0 
    if t in q:
        t_num = 1
    c_num = 0
    for c in chars:
        for q in query:
            if c == q:
                c_num += 1
    s_num = 0
    for s in chars:
        for q in query:
            if s == q:
                s_num += 1
                
    #Our formula:
    sim_dict[idx] = cosine_similarity_dict(query_vec, desc) * 0.25 +  t_num * 0.25 + c_num * 0.25 + s_num * 0.25
    
sim_ds = pd.DataFrame({'similarity': sim_dict.values()})
ds['similarity'] = sim_dict.values()
k = 4
clean_dict = {k: sim_dict[k] for k in sim_dict.keys() if sim_dict[k] != 0.0 and not np.isnan(sim_dict[k])}
clean_dict = {k: v for k, v in sorted(clean_dict.items(), key=lambda item: item[1])}

l = [i for i in clean_dict.keys()]
l = l[-k:]

heap = hq.heapify(l)
ds.iloc[l, [-1, 9, 14, 15]].sort_values('similarity', ascending=False)

saiyan race movie goku


<ipython-input-35-c07044aa0f89>:16: RuntimeWarning: invalid value encountered in double_scalars
  return c / (norm1 * norm2)


,similarity,anime_description,url,title
1437,0.099594,"Bardock, Son Goku's father, is a low-ranking S...",https://myanimelist.net/anime/986/Dragon_Ball_...,Dragon Ball Z Special 1: Tatta Hitori no Saish...
5547,0.093649,"One peaceful afternoon, the Son family and fri...",https://myanimelist.net/anime/22695/Dragon_Bal...,Dragon Ball Z: Summer Vacation Special
4890,0.090904,Specials included with the original series.\n\...,https://myanimelist.net/anime/2520/Dragon_Ball...,Dragon Ball Specials
3158,0.089819,A retelling of Dragon Ball's origin with a dif...,https://myanimelist.net/anime/2748/Dash_Kappei,Dash! Kappei


We wanted to find a movie related to the sayian race that starred Goku as a character. The results seem acceptable 

## 4. BONUS: Understanding the anime's reviews

We start by parsing again the html, this time considering the reviews 


In [123]:
def review_parser2(article_path, soup):
        anime_title = soup.title.text.strip()
        anime_type = ""
        try:
            anime_type = getText(soup.find('span', text="Type:").parent.a)
        except:
            pass
        anime_num_episodes = ""
        try:
            anime_num_episodes = getText(soup.find('span', text="Episodes:").parent)
        except:
            pass
        anime_aired = ""
        try:
            anime_aired = getText(soup.find('span', text="Aired:").parent).split(" to ")
        except:
            pass
        anime_status = ""
        try:
            anime_status = getText(soup.find('span', text="Status:").parent)
        except:
            pass
        anime_score = ""
        try:
            anime_score = getText(soup.find('span', text="Score:").parent.find_all('span', {'class': 'score-label'})[0])
        except:
            pass
        anime_users = ""
        try:
            anime_users = getText(soup.find('span', text="Members:").parent)
        except:
            pass
        anime_rank = ""
        try:
            anime_rank = getText(soup.find('span', text="Ranked:").parent)
        except:
            pass
        anime_popularity = ""
        try:
            anime_popularity = getText(soup.find('span', text="Popularity:").parent)
        except:
            pass
        anime_description = ""
        try:
            anime_description = getText(soup.find('h2', text="Synopsis").parent.parent.p)
        except:
            pass
        anime_related = []
        try: 
            anime_related = list(set(map(getText,soup.find('h2', text="Related Anime").parent.parent.tr.find_all(lambda tag:tag.name == "a" and tag.href != "")))) 
        except: 
            pass
        anime_characters = []
        try:
            anime_characters = list(filter(None,list(map(lambda tr: re.split('\n+', tr.find_all('td')[1].text.strip())[0], soup.find('h2', text="Characters & Voice Actors").find_next('div').find_all('tr')))))
            anime_characters = [e.split(", ") for e in anime_characters]
        except:
            pass
        anime_voices = []
        try:
            anime_voices = list(filter(None,list(map(lambda tr: re.split('\n+', tr.find_all('td')[0].text.strip())[0], soup.find('h2', text="Characters & Voice Actors").find_next('div').find_all('tr')))))
            anime_voices = [e.split(", ") for e in anime_voices]
        except:
            pass
        anime_staff = []
        try:
            anime_staff = list(map(lambda tr: [re.split("\n+", tr.text.strip())[0].split(', '), re.split("\n+", tr.text.strip())[1]], soup.find('h2', text="Staff").find_next('div').find_all('tr')))
        except:
            pass
        try:
            anime_review = [x.strip() for x in re.split('(<br>|\n+)', soup.find('a', text="More reviews").find_next('div').text) if len(x) > 100]
        except:
            pass
        
        l = [anime_title,
                anime_type,
                anime_num_episodes,
                anime_aired,
                anime_status,
                anime_score,
                anime_users,
                anime_rank,
                anime_popularity,
                anime_description,
                anime_related,
                anime_characters,
                anime_voices,
                anime_staff,
                anime_review]
        return l

In [ ]:
num_of_docs = 19125
i = 1
with open("anime.csv", 'rb') as urls:
    urls_ds = pd.read_csv(urls)
    with open("anime_review2.tsv", 'w', encoding="utf-8", newline='') as anime:
        for p in range(1, num_of_pages + 1):
            flag = True
            if (flag): 
                article_path = page_dir + str(p) + "/" + article + "_" + str(i) + ".html"
                while os.path.isfile(article_path):
                    article_path = page_dir + str(p) + "/" + article + "_" + str(i) + ".html"
                    if os.path.isfile(article_path):
                        if i % 100 == 0:
                            print(p, "--->", i, "--->", article_path)
                        with open(article_path, 'rb') as html:
                            soup = BeautifulSoup(html, "html.parser")
                            l = review_parser2(article_path, soup)
                            l.append(urls_ds.url[i - 1])
                            l.append(urls_ds.name[i - 1])
                            writer = csv.writer(anime, delimiter='\t')
                            writer.writerow(l)
                    i += 1

DISCLAIMER: We actually made a mistake here, as while parsing we only gathered one review for each anime. This probably brings innacurate results.

Once again, we add columns to the dataframe

In [141]:
review_ds2 = pd.read_csv("anime_review2.tsv", delimiter='\t')

review_ds2.columns = ['anime_title',
                'anime_type',
                'anime_num_episodes',
                'anime_aired',
                'anime_status',
                'anime_score',
                'anime_users',
                'anime_rank',
                'anime_popularity',
                'anime_description',
                'anime_related',
                'anime_characters',
                'anime_voices',
                'anime_staff',
                'review',
                'url',
                'title']

len(ds)

18741

We only consider the first three sentence of a review. To do so, we check the first three stopwords in a review, and discard everything that comes after it.

In [ ]:
review_ds2['review_short'] = review_ds2['review'].apply(lambda x: x.split(".")[:3])
review_ds2['review_short_string'] = [''.join(map(str, l)) for l in review_ds['anime_review_short']]
review_ds2.drop(columns=['review', 'review_short'])
review_ds2.head(10)

To calculate if a review is positive or not, we use the Vader module from NLTK and its polarity scores measure, which can roughly check if a review is positive or not. The polarity score returns three values, 'negative', 'neutral', and 'positive', referring to the analyzed text. We then use the compound function to normalize the three scores into a single value. The compound score is contained between -1 and 1: -1 means that the review was absolutely negative, 0 that the review was neutral, 1 that it was absolutely positive.

IMPORTANT: We didn't preprocess the reviews since Vader actually takes into consideration capitalization and exclamation.

In [146]:
review_ds2['scores'] = review_ds2['review_short_string'].apply(lambda review: vader.polarity_scores(review))

In [148]:
review_ds2['compound'] = review_ds2['scores'].apply(lambda score_dict: score_dict['compound'])

- What Anime Type are the most popular based on your analysis?

In [316]:
type_df = review_ds2.groupby('anime_type').mean()
type_df_sorted = type_df.sort_values(by = ['compound'], ascending=False);

In [317]:
type_df_sorted.drop('anime_score', axis=1)


,compound
anime_type,
TV,0.212936
Special,0.192303
OVA,0.152073
Movie,0.142555
ONA,0.104817


We can see that from our analysis the animes classified as Tv shows are (marginally) the ones with the most positive reviews, quickly followed by the "special" category.

 - Does the number of episodes has any impact on the sentiment of the users?

Here we check with a simple Pearson coefficient if we can find a correlation between the number of episodes and the sentiment of the users

In [259]:
new_df_corr_new = review_ds2[['compound', 'anime_num_episodes']]
c = new_df_corr_new[(new_df_corr_new.compound != 0) & (new_df_corr_new.anime_num_episodes != 'Unknown')]
#we filtered for the compound values different than 0, since if an anime doesn't have any review it will
#display 0 as a compound score and that would skew the results given the amount of animes without reviews

In [266]:
pd.options.mode.chained_assignment = None #to avoid errors when converting to int

In [267]:
c['anime_num_episodes_int'] = c['anime_num_episodes'].astype(int) #converting to the correct type

In [270]:
coeff = pearsonr(c['anime_num_episodes_int'], c['compound'])
coeff[0]

0.028011316551619578

There seems to be no correlation. This felt strange to us and it may be that our analysis was flawed.

- Is your sentiment analysis in line with the scores of the animes? 

In similar fashion, we repeat the steps we did earlier and check for a correlation between the score of an anime and its compound score

In [ ]:
d = review_ds2[['compound', 'anime_score']]
e = review_ds2[(d.compound != 0)]


In [298]:
f = e.dropna(axis= 0, how= 'any');

In [300]:
coeff = pearsonr(f['anime_score'], f['compound'])
coeff[0]

0.10796921928967673

Again, no correlation. We feel this shouldn't actually be the case and the results are probably wrong due to the fact that we didn't parse the reviews in the intended way.

---------------------------------------------------------------------

## 5. Algorithmic question

In this problem, the goal is to maximize the sum of values contained in a list, where each value represents the time that the person who made the appointment wants to spend with the personal trainer, so you want to provide the personal trainer with a program that maximizes the total length of accepted appointments. The important condition is that the personal trainer needs a break between appointments and cannot accept two consecutive bookings. In addition, reservations are considered in the order in which they were made, so that the personal trainer cannot go back and accept past bookings, but must follow the chronological order.



### **5.1**

### **Formalization of the problem**
From an algorithmic point of view, the problem results in: given an array of positive numbers, find the maximum sum of a subsequence with the constraint that no two numbers in the sequence should be adjacent in the array.

### **Alghorithmic Idea**
The time complexity of this solution is $O(n)$

### **PseudoCode**

	Input:
  	  Arr = array of positive numbers with length n.

	Function trainer(Arr):
		n = length of Arr.
		Base Cases
		In case there are no bookings, no operations are carried out.
		return 0
		In case there is only one booking, only this one is chosen.
		return Arr[0]

		Advanced Case:
		Define an empty list and Start for cycle.
	
		Start while cycle.
		Extract the indices of the numbers that composed the largest sum.

		Print the sub-list that generates the maximum sum and the maximum
		corresponding sum.

### **5.2** 
Implement a program that given in input an instance in the form given above, gives the optimal solution.

In [125]:
def trainer(Arr):
    n = len(Arr)
    if not Arr:
        return 0
    if n == 1:
        return Arr[0]

    new_list = [None for _ in range(n + 1)]
    new_list[0] = 0
    new_list[1] = Arr[0]


    for i in range(2, n + 1):
        new_list[i] = max(new_list[i - 2] + Arr[i - 1], new_list[i - 1])

    res = []
    i = n
    while i > 0:
        if i > 1:
            if new_list[i - 1] > new_list[i - 2] + Arr[i - 1]:
                i -=  1
            else:
                res.append(Arr[i - 1])
                i -=  2
        elif i == 1 and new_list[i - 1] < Arr[i - 1]:
            res.append(Arr[i - 1])
            i -= 1
        else:
            i -= 1
    res.reverse()
    return res, sum(res)

#### Different Tests

In [126]:
## Test 01
Arr = [30, 40, 25, 50, 30, 20]
print('The appointment list is:', Arr, '\n') 
print('The best way to accept appointements is:', trainer(Arr)[0], 'with max duration of:', trainer(Arr)[1])

The appointment list is: [30, 40, 25, 50, 30, 20] 

The best way to accept appointements is: [40, 50, 20] with max duration of: 110


In [127]:
## Test 02
Arr = [30, 40, 25, 10, 20, 50, 30, 20]
print('The appointment list is:', Arr, '\n') 
print('The best way to accept appointements is:', trainer(Arr)[0], 'with max duration of:', trainer(Arr)[1])

The appointment list is: [30, 40, 25, 10, 20, 50, 30, 20] 

The best way to accept appointements is: [30, 25, 50, 20] with max duration of: 125


In [128]:
## Test 03 (n = 50)
Arr = [79, 112, 28, 33, 117, 30, 41, 23, 7, 58, 113, 12, 78, 43, 92, 75, 13, 98, 83, 102, 1, 15, 52, 31, 88, 71, 70, 3, 66, 26, 96, 20, 47, 36, 110, 62, 18, 93, 67, 84, 48, 6,
       10, 46, 37, 72, 34, 51, 89, 40]
print('The appointment list is:', Arr, '\n') 
print('The best way to accept appointements is:', trainer(Arr)[0], 'with max duration of:', trainer(Arr)[1])

The appointment list is: [79, 112, 28, 33, 117, 30, 41, 23, 7, 58, 113, 12, 78, 43, 92, 75, 13, 98, 83, 102, 1, 15, 52, 31, 88, 71, 70, 3, 66, 26, 96, 20, 47, 36, 110, 62, 18, 93, 67, 84, 48, 6, 10, 46, 37, 72, 34, 51, 89, 40] 

The best way to accept appointements is: [112, 117, 41, 7, 113, 78, 92, 98, 102, 52, 88, 70, 66, 96, 47, 110, 93, 84, 6, 46, 72, 51, 40] with max duration of: 1681
